In [1]:
# -*- coding: utf-8 -*-
import os
import sys
from readmdict import MDX
from glob import glob

# 加载.mdx文件,修改此处，可以更换不同的mdx；
# 注意： 路径下必须同时包含mdx和css文件。
dict_dir = 'Vocabulary/新东方英语词根词缀'
#dict_dir = 'Vocabulary/etymology'

mdx_name = glob(os.path.join(dict_dir,'*.mdx'))[0]
css_name = glob(os.path.join(dict_dir,'*.css'))[0]

print('mdx_name = ', mdx_name)
print('css_name = ', css_name)
css_name = os.path.basename(css_name)

mdx = MDX(mdx_name)
headwords = [*mdx]       # 单词名列表
items = [*mdx.items()]   # 释义html源码列表
n = len(headwords)
m = len(items)
assert n == m



    
wordIndex  = 0
word,html = items[wordIndex]
#print('********Search: ', wordIndex,word, html)
result = html.decode()
result = result.replace(css_name,os.path.join('static/css',css_name))





mdx_name =  Vocabulary/新东方英语词根词缀/NewOrientalRootsAffixes.mdx
css_name =  Vocabulary/新东方英语词根词缀/NewOrientalRootsAffixes.css


In [16]:
data = [*mdx.items()][5][1].decode('utf-8')
print(data)
#decoded_data = {key.decode('utf-8'): value.decode('utf-8') for key, value in data.items()}
#print(data)

from bs4 import BeautifulSoup

text = data

soup = BeautifulSoup(text, 'html.parser')

# 找到所有词条
entries = soup.find_all('div', class_='word-entry')

# 打印提取的信息
for entry in entries:
    entry_id = entry.get('id')
    word = entry.find('span', class_='bold').get_text()
    word_type = entry.find('span', class_='textItalic').get_text()
    meaning = entry.find('p', class_='meaning').get_text(strip=True)

    print(f"词条ID: {entry_id}\n单词: {word}\n词性: {word_type}\n释义: {meaning}\n")


<div class="MdxNewOrientalRootsAffixes">
<div id="TopOfPage_NewOrientalRootsAffixes" class="golden-dict-padding"></div>
<link rel="stylesheet" type="text/css" href="NewOrientalRootsAffixes.css"><h2 onclick="SwitchMore(event)" onmousedown="SwitchMoreMouseDown(event)" class="sigil_not_in_toc"><span class="item-num">43. </span>-ed 表形容词</h2>

  <hr><div class="head-words"><a onclick="SwitchHeadWord(event)" href="#NewOrientalHeadWord_145_aged">aged</a><span class="word-delimiter">,</span><a onclick="SwitchHeadWord(event)" href="#NewOrientalHeadWord_145_bearded">bearded</a><span class="word-delimiter">,</span><a onclick="SwitchHeadWord(event)" href="#NewOrientalHeadWord_145_coloured">coloured</a><span class="word-delimiter">,</span><a onclick="SwitchHeadWord(event)" href="#NewOrientalHeadWord_145_gifted">gifted</a><span class="word-delimiter">,</span><a onclick="SwitchHeadWord(event)" href="#NewOrientalHeadWord_145_pointed">pointed</a><span class="word-delimiter">,</span><a class="key-head-w